In [78]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
import sys
import json
if "../../src" not in sys.path:
    sys.path.append("../../src")
import utils
import settings
from opensearch_client import OpenSearchClient
from pdf_processor import PdfProcessor

wrapper = utils.get_text_wrapper()

In [80]:
INDEX_NAME = "unstructured-knn-index"

Get some pdfs

In [81]:
pdfs = [
    {
        "file_path": settings.DATA_DIR / "admitere" / "Test limba straina 21 03 detalii S1 2025.pdf",
        "url": "https://admitere.pub.ro/docs/Test%20limba%20straina%2021%2003%20detalii%20S1%202025.pdf",
        "output_dir": settings.DATA_DIR / "content" / "admitere",
    }
]

In [82]:
pdf = pdfs[0]

file_path, url = pdf["file_path"], pdf["url"]

pdfProcessor = PdfProcessor(file_path, url=url)

pdfProcessor.partition_pdf()

pdfProcessor.cleanup_elements()

pdfProcessor.perform_chunking()

In [83]:
for chunk in pdfProcessor.chunks:
    print(wrapper.fill(json.dumps(chunk, indent=4, ensure_ascii=False)))
    print(wrapper.fill("-" * wrapper.width))


{
    "id": "9cf23be3e6b1bcbd56f0cab3918a505c41688bb641a64cac3a8509bc9628e5d1-1",
    "text": "Varianta ; Cum dovedesc
la înscriere ; Este necesar să susțin testul de limbă POLITEHNICA Bucureşti? Liceu absolvit în respectiva limbă de
predare ; Dacă acest lucru nu apare evident din celelalte documente furnizate la înscriere, am grijă să încarc o copie a
unui document eliberat de liceu şi din care să reiasă că am urmat un program de studii în respectiva limbă ; NU ;
Certificat de competențe lingvistice pentru respectiva limbă de predare (minim B2); ; Încarc o copie a certificatului ;
NU ;  Disciplină de tip limbă străină promovată în liceu cu media cel puțin 8,00 (pentru clasele a IX- a,a X-a, a XI-a).
; Încarc o copie a unui document eliberat de liceul urmat din care reiese acest lucru ; NU ;  Proba A, B sau C de la
examenul de Bacalaureat; ; Încare o copie completă a diplomei de Bacalaureat (inclusiv situaţia la examen) ; NU ;  Aleg
să susțin testul de limbă POLITEHNICA Bucureşti; ; Nu

Ingest data into OpenSearch

In [84]:
data = pdfProcessor.format_data(INDEX_NAME)

In [85]:
client = OpenSearchClient()

In [86]:
ret = client.ingest_data_bulk(data)

In [87]:
print(client.get_all_elements(INDEX_NAME))

{'took': 893, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'unstructured-knn-index', '_id': '9cf23be3e6b1bcbd56f0cab3918a505c41688bb641a64cac3a8509bc9628e5d1-1', '_score': 1.0, '_source': {'filename': 'Test limba straina 21 03 detalii S1 2025.pdf', 'page_number': 1, 'table_as_markdown': None, 'id': '9cf23be3e6b1bcbd56f0cab3918a505c41688bb641a64cac3a8509bc9628e5d1-1', 'text': 'Despre testul de limbă on-line POLITEHNICA București din data de 21 martie 2025 L Ce este cu acest test? Am nevoie de el? Dacă doresc să beneficiez de avantajele studiului într-una din limbile de largă circulație (engleză, franceză, germană) și în lista mea de opțiuni includ domenii la care școlarizarea se desfășoară într-una din acest limbi, trebuie, conform legii, să fac dovada că am cunoștințele lingvistice necesare. Dovada se face folosind oricare dintre variantele de mai jos:', '

In [88]:
for obj in ret:
    print(json.dumps(obj, indent=4))

[
    true,
    {
        "index": {
            "_index": "unstructured-knn-index",
            "_id": "9cf23be3e6b1bcbd56f0cab3918a505c41688bb641a64cac3a8509bc9628e5d1-1",
            "_version": 3,
            "result": "updated",
            "_shards": {
                "total": 2,
                "successful": 2,
                "failed": 0
            },
            "_seq_no": 4,
            "_primary_term": 1,
            "status": 200
        }
    }
]
[
    true,
    {
        "index": {
            "_index": "unstructured-knn-index",
            "_id": "9cf23be3e6b1bcbd56f0cab3918a505c41688bb641a64cac3a8509bc9628e5d1-1",
            "_version": 4,
            "result": "updated",
            "_shards": {
                "total": 2,
                "successful": 2,
                "failed": 0
            },
            "_seq_no": 5,
            "_primary_term": 1,
            "status": 200
        }
    }
]
[
    true,
    {
        "index": {
            "_index": "unstructur

In [89]:
print(json.dumps(data, indent=4, ensure_ascii=False))

[
    {
        "_index": "unstructured-knn-index",
        "_id": "9cf23be3e6b1bcbd56f0cab3918a505c41688bb641a64cac3a8509bc9628e5d1-1",
        "id": "9cf23be3e6b1bcbd56f0cab3918a505c41688bb641a64cac3a8509bc9628e5d1-1",
        "text": "Varianta ; Cum dovedesc la înscriere ; Este necesar să susțin testul de limbă POLITEHNICA Bucureşti? Liceu absolvit în respectiva limbă de predare ; Dacă acest lucru nu apare evident din celelalte documente furnizate la înscriere, am grijă să încarc o copie a unui document eliberat de liceu şi din care să reiasă că am urmat un program de studii în respectiva limbă ; NU ;  Certificat de competențe lingvistice pentru respectiva limbă de predare (minim B2); ; Încarc o copie a certificatului ; NU ;  Disciplină de tip limbă străină promovată în liceu cu media cel puțin 8,00 (pentru clasele a IX- a,a X-a, a XI-a). ; Încarc o copie a unui document eliberat de liceul urmat din care reiese acest lucru ; NU ;  Proba A, B sau C de la examenul de Bacalaureat; ; În